In [6]:
%run ./read_file.ipynb 

In [7]:
df_labels = readCheckpoint('labels')
#create a timestamp column and drop the old timestamp - does not appear necessary
df_labels = df_labels.withColumn('ts2', to_timestamp(col('timestamp'))).drop('timestamp')
#check new col data types
df_labels.printSchema()

AnalysisException: Path does not exist: s3a://sapient-bucket-trusted/dev/labels

In [ ]:
def build_adjaceny(df, dataset = 'ecar'):
    
    if dataset == 'labels':
        df = df.sort(col("ts2").asc())
    else : 
        #first step sort the data and check
        df = df.sort(col("timestamp").asc())
    
    #now get overlapping connections

    #build collections of each type of ID
    #TODO make this one select statement
    col_objID = df.select("objectID").distinct().collect()
    col_actID = df.select("actorID").distinct().collect()

    #put these columns into a dictionary and turn into a spark data frame
    data = [{"actorIDs" : [row.actorID for row in col_actID], \
                   "objectIDs" : [row.objectID for row in col_objID]}]

    id_intersect = spark.createDataFrame(data)

    #compare ID pools to find actIDs that are nodes
    id_intersect = id_intersect.withColumn("id_int", \
                                           array_intersect(col("actorIDs"),col("objectIDs")))

    #now build a list of the nodal actIDs and print the length
    actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
    
    #create ref dict
    dict_node_edge_events = {}
    
    #for all actors in actIDs
    for aID in actIDs: 
        first_id = df.where((col("objectID") == aID) \
                               & (col("actorID")!= aID)).limit(1).collect()[0]['id']
        
        #now add this to the dictionary as a key
        first_event = str(aID+':'+first_id)
        dict_node_edge_events[first_event] = []
        
        #now add all the events where the actorID is present but not the objectID, group by unique object ID
        #and pull out the first event - and I checked and believe the sorting holds through grouping
        col_select_children_events = df.select('objectID','id')\
        .where((col("actorID") == aID) & \
                 (col("objectID") != aID)).groupBy("objectID").agg(first("id")).collect() 
        
        #add children objects into dictioinary 
        dict_node_edge_events[first_event] = [str(row["objectID"]+':'+row["first(id)"]) for row in col_select_children_events]
        
    return dict_node_edge_events